In [1]:
with open("../backend/logging/search.log", "r") as f, open("pre_file.csv", "w") as f2:
    for line_ in f.readlines():
        try:
            time, event, query, documents_ = line_.split(";")
            time, event, query, documents_ = time, event.strip(), query.strip(), documents_.strip()
            if event == "Search.viewed":
                documents = documents_.split("|")

                for document in documents:
                    f2.write(f"{query},{document},1,0\n")
            else:
                f2.write(f"{query},{documents_},0,1\n")
        except:
            print("error", line_)

In [2]:
!sort -t ',' -k 1n pre_file.csv > pre_file_sorted.csv

In [3]:
!rm pre_file.csv

In [4]:
now_q = ""
now_d = ""

v_c = (0,0)

first = True

with open("pre_file_sorted.csv", "r") as f3, open("features_file.csv", "w") as f4:
    for line_ in f3.readlines():
        try:
            list_all = [x.strip() for x in line_.split(",")]

            click, view, document = list_all.pop(),list_all.pop(),list_all.pop()
            query = ",".join(list_all)

            if first:
                now_q = query
                now_d = document
                v_c = (int(view),int(click))
                first = False

            if (now_q, now_d) != (query,document):
                f4.write(f"{now_q},{now_d},{v_c[0]},{v_c[1]}\n")
                now_q = query
                now_d = document
                v_c = (int(view),int(click))
            else:
                v_c = (v_c[0] + int(view), v_c[1] + int(click))
        except:
            print("error", line_)

In [5]:
import numpy as np
with open("features_file.csv", "r") as f5, open("reference_colletion.csv", "w") as f6:

    f6.write(f"query,document,label\n")

    for line_ in f5.readlines():

        list_all = [x.strip() for x in line_.split(",")]

        click, view, document = list_all.pop(),list_all.pop(),list_all.pop()
        query = " ".join(list_all)

        view, click = int(view), int(click)
        prior_a = 5
        prior_b = 25
        posterior_a = click+prior_a
        posterior_b = prior_b+(view-click)
        label = np.round(((posterior_a  / (posterior_b+posterior_a))*32)).astype("int")
        f6.write(f"{query},{document},{label}\n")